vector rag local cos similarity  and rag

In [127]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
from langchain.chat_models import ChatOpenAI
import os

In [129]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
)


In [130]:
model.predict("2024 Fall of Damascus")

'As of my last knowledge update in October 2023, there was no specific event known as the "Fall of Damascus" in 2024. If you are referring to a hypothetical situation, a fictional narrative, or a recent development that has emerged after my last update, I would not have that information.\n\nIf you\'re looking for information on the historical significance of Damascus or its geopolitical situation, please provide more context, and I\'d be happy to help!'

In [131]:
import wikipedia
content = wikipedia.page("Fall of Damascus").content

with open("damascus.txt","w") as file:
    file.write(content)


In [132]:
with open("damascus.txt","r") as file:
    content = file.read()

content[:10]

'On 7 Decem'

In [133]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap = 56,
    length_function=len
)

In [134]:
chunks = textSplitter.create_documents([content])

In [135]:
chunks[0].page_content

'On 7 December 2024, the Syrian opposition group known as the Southern Operations Room led forces that entered the Rif Dimashq region of Syria from the south, and those forces then came within 20 kilometres (12 mi) of the capital Damascus. The Syrian Army'

In [136]:
chunks = textSplitter.create_documents([content])
for i, chunk in enumerate(chunks):
    print(f"{i+1}. chunk ")
    print(chunk.page_content)
    print("")

1. chunk 
On 7 December 2024, the Syrian opposition group known as the Southern Operations Room led forces that entered the Rif Dimashq region of Syria from the south, and those forces then came within 20 kilometres (12 mi) of the capital Damascus. The Syrian Army

2. chunk 
(12 mi) of the capital Damascus. The Syrian Army withdrew from multiple points in the outskirts. Concurrently with the advance towards Damascus, opposition militia Tahrir al-Sham and the Turkish-backed Syrian National Army in the north launched an

3. chunk 
Syrian National Army in the north launched an offensive into Homs, while the Syrian Free Army advanced into the capital from the southeast. By 8 December 2024, rebel forces entered the city's Barzeh neighborhood. According to official state reports in

4. chunk 
neighborhood. According to official state reports in Russian mass media and media footage, President Bashar al-Assad left Damascus by air to Moscow, where he was granted asylum, sealing the fall of his 

chunk embedding 

In [137]:
import openai
import uuid

In [138]:
def generateEmbedding(text):
    emb =openai.embeddings.create(input=text,model="text-embedding-ada-002")
    return emb.data[0].embedding

In [139]:
chunkEmbeddings = []

for i,chunk in enumerate(chunks):
    #print(i+1)
    chunkEmbeddings.append(
        {
            "id":uuid.uuid1(),
            "content": chunk.page_content,
            "contentVector": generateEmbedding(chunk.page_content)
        }
    )

In [140]:
chunkEmbeddings[0]

{'id': UUID('8a0bbd5b-c2ff-11ef-bda5-d49390092826'),
 'content': 'On 7 December 2024, the Syrian opposition group known as the Southern Operations Room led forces that entered the Rif Dimashq region of Syria from the south, and those forces then came within 20 kilometres (12 mi) of the capital Damascus. The Syrian Army',
 'contentVector': [-0.027151992544531822,
  -0.00937675405293703,
  -0.04512022063136101,
  -0.015599085949361324,
  -0.029121842235326767,
  0.03982292115688324,
  -0.017835132777690887,
  -0.01735597848892212,
  -0.028456352651119232,
  -0.012225051410496235,
  0.02773762308061123,
  0.015226410701870918,
  -0.00947657786309719,
  -0.016557391732931137,
  -0.0036069557536393404,
  -0.0044687651097774506,
  -0.0023708080407232046,
  -0.02118920162320137,
  0.008644714951515198,
  -0.013642544858157635,
  0.0032742107287049294,
  -0.0226932093501091,
  0.0008534908993169665,
  -0.013256561011075974,
  0.010048898868262768,
  0.0001480715291108936,
  0.00190330133773386

Retriavel

In [141]:
query = "What is the most recent date on which Damascus was captured by the rebels?"
model.predict(query)

'As of my last knowledge update in October 2023, Damascus has not been captured by rebel forces in the ongoing Syrian civil war. The city has largely remained under the control of the Syrian government, although there have been various clashes and shifts in control in surrounding areas. The situation in Syria is complex and fluid, so for the most current information, I recommend checking the latest news updates.'

In [142]:
queryEmbed = generateEmbedding(query)

In [143]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def getVectorSimilarity(l1,l2):
    v1 = np.array([l1])
    v2 = np.array([l2])

    return cosine_similarity(v1,v2)[0][0]



In [144]:
def retrievalData(queryEmbed,topk=15):
    similarityCollection = []

    for chunkEmbedding in chunkEmbeddings:

        similarity = getVectorSimilarity(queryEmbed,chunkEmbedding["contentVector"])

        similarityCollection.append(
                                    {
                                        "similarityScore": similarity,
                                        "context": chunkEmbedding["content"]
                                    }
                                    )
                                    
    similarityCollection = sorted(similarityCollection, key=lambda x: x["similarityScore"],reverse=True)
    return similarityCollection[:topk]
    
    


        


In [145]:
retrievaledData=retrievalData(queryEmbed)

In [146]:
" ".join(data["context"]  for data in retrievaledData)

'On 7 December 2024, Syrian rebels announced that they started surrounding Damascus after capturing nearby towns, with rebel commander Hassan Abdel Ghani stating: "Our forces have begun implementing the final phase of encircling the capital Damascus." The By the beginning of 7 December 2024, rebels had captured most of Daraa Governorate and Suwayda Governorate and granted safe passage to some pro-government forces towards Damascus. Maher al-Assad, Bashar\'s brother and head of the 4th Armoured Division, was On 7 December 2024, the Syrian opposition group known as the Southern Operations Room led forces that entered the Rif Dimashq region of Syria from the south, and those forces then came within 20 kilometres (12 mi) of the capital Damascus. The Syrian Army final phase of encircling the capital Damascus." The rebels started encircling the capital after capturing Al-Sanamayn, a town 20 kilometres (12 mi) from the southern entrance of Damascus. In the Rif Dimashq region, pro-government f

Augmented Generation

In [147]:
model.predict(query)

'As of my last update in October 2023, Damascus has not been captured by rebel forces during the Syrian Civil War. While there have been significant battles and periods of intense fighting in and around the city, the Syrian government has maintained control over Damascus throughout the conflict. If there have been any developments after October 2023, I would recommend checking the latest news sources for the most current information.'

In [148]:
from langchain import PromptTemplate

ragTemplate = """
###
CONTEXT
###
'''
{context}
'''

###
RULES:
Learn from the CONTEXT and answer the following question.
Provide your answer directly in the first sentence. 

###

QUESTION:
{question}

"""


In [149]:
ragPromt = PromptTemplate(
    template = ragTemplate,
    input_variables = ["context","question"]
)

In [150]:
prompt = ragPromt.format(question=query, context=" ".join(data["context"]  for data in retrievaledData))

In [151]:
model.predict(prompt)

'The most recent date on which Damascus was captured by the rebels was 8 December 2024.'